# GEMM on GPU

## 1. Set-up

In [1]:
!pip install tlcpack-nightly-cu102 -f https://tlcpack.ai/wheels
!pip install "numpy<2.0.0"

Looking in links: https://tlcpack.ai/wheels
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.5/428.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 13.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2


## 3. Check the implementation of `make_gemm_gpu_scheduler` function in `src.ops`

The function implements General Matrix Multiply (GEMM) on GPU. You should use TVM to optimize it.

Let $A \in \mathbb{R}^{m \times k}$, $W \in \mathbb{R}^{k \times n}$, and $B \in \mathbb{R}^{m \times n}$, then
$$
B = A \times W
$$
Please see the numpy matmul function for more detail: [link](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

The `make_gemm_gpu_scheduler` takes $m$, $k$, and $n$. The first matrix is $m \times k$, the second matrix is $k \times n$, and the output matrix is $m \times n$.

The function returns both the TVM scheduler and the TVM opterator for
1. Input $a$
2. Input $w$
3. Output $b$

The scheduler should be able to used to build a function with signature $func(a, w, b)$.
Please see the following cells for usage.

In [15]:
import time
import tvm
import numpy as np
from tvm import te

# benchmark for tvm implementation
def benchmark_gemm_tvm(schedule_func, M, K, N, device, a_np, w_np, num_runs=30, repeat=20):
    s, A, W, B = schedule_func(M, K, N)
    func = tvm.build(s, [A, W, B], target="cuda")

    dev = tvm.cuda(0)
    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    out_tvm = tvm.nd.array(np.zeros((M, N), dtype), dev)
    evaluator = func.time_evaluator(func.entry_name, dev, number=num_runs, repeat=repeat)
    cost = evaluator(a, w, out_tvm).mean

    return cost, out_tvm.asnumpy(), func, (s, A, W, B)

def base_declaration(M, K, N):
    k = te.reduce_axis((0, K), "k")
    A = te.placeholder((M, K), name="A")
    B = te.placeholder((K, N), name="B")
    C = te.compute((M, N), lambda x, y: te.sum(A[x, k] * B[k, y], axis=k), name="C")
    s = te.create_schedule(C.op)
    return k, s, A, B, C

M = 1024
N = 512
K = 2048
dtype = 'float32'
a_np = np.random.rand(M, K).astype(dtype)
w_np = np.random.rand(K, N).astype(dtype)
ref = np.matmul(a_np, w_np)


In [ ]:
def make_gemm_gpu_scheduler_naive(M, K, N, verbose=True):
    k, s, A, B, C = base_declaration(M, K, N)

    # overall index of a thread: 𝑖=blockIdx.x×blockDim.x+threadIdx.x
    block_x = te.thread_axis("blockIdx.y")
    block_y = te.thread_axis("blockIdx.x")

    x, y = s[C].op.axis
    (k,) = s[C].op.reduce_axis
    s[C].bind(y, block_y)
    s[C].bind(x, block_x)
    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, B, C], simple_mode=True))
        print("=" * 100)
    return s, A, B, C

# naive TVM
dev = tvm.cuda()
naive_time, naive_res, naive_func, naive_comp = benchmark_gemm_tvm(
    make_gemm_gpu_scheduler_naive, M, K, N, dev, a_np, w_np, num_runs=5, repeat=5
)
np.testing.assert_allclose(naive_res, ref, rtol=1e-4)
print(f"[TVM Naive] time: {naive_time*1e3:.4f} ms")

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_y = T.launch_thread("blockIdx.y", 1024)
        blockIdx_x = T.launch_thread("blockIdx.x", 512)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_y * 512 + blockIdx_x] = T.float32(0)
        for k in range(2048):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_y * 512 + blockIdx_x] = C_1[blockIdx_y * 512 + blockIdx_x] + A_1[blockIdx_y * 2048 + k] * B_1[k * 512 + blockIdx_x]
[TVM Naive] time: 84.5233 ms


In [ ]:
# opt v1: tiling + threads 1D
def make_gemm_gpu_scheduler_v1(M, K, N, verbose=True):
    k, s, A, B, C = base_declaration(M, K, N)

    x, y = s[C].op.axis

    # split the axes
    xo, xi = s[C].split(x, factor=32)

    # bind the outer axes to blocks
    s[C].bind(xo, te.thread_axis("blockIdx.x"))
    s[C].bind(y, te.thread_axis("blockIdx.y"))

    # bind the inner axes to threads
    s[C].bind(xi, te.thread_axis("threadIdx.x"))

    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, B, C], simple_mode=True))
        print("=" * 100)

    return s, A, B, C

# Test the v1 optimization
dev = tvm.cuda()
v1_time, v1_res, v1_func, v1_comp = benchmark_gemm_tvm(
    make_gemm_gpu_scheduler_v1, M, K, N, dev, a_np, w_np, num_runs=20, repeat=20
)
np.testing.assert_allclose(v1_res, ref, rtol=1e-4)
print(f"[TVM v1] time: {v1_time*1e3:.4f} ms")

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 32)
        threadIdx_x = T.launch_thread("threadIdx.x", 32)
        blockIdx_y = T.launch_thread("blockIdx.y", 512)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y] = T.float32(0)
        for k in range(2048):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y] = C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y] + A_1[blockIdx_x * 65536 + threadIdx_x * 2048 + k] * B_1[k * 512 + blockIdx_y]
[TVM v1] time: 36.9797 ms


In [ ]:
# opt v2: tiling + threads 2D
def make_gemm_gpu_scheduler_v2(M, K, N, verbose=True):
    k, s, A, B, C = base_declaration(M, K, N)

    x, y = s[C].op.axis

    # split the axes
    xo, xi = s[C].split(x, factor=32)
    yo, yi = s[C].split(y, factor=32)

    # bind the outer axes to blocks
    s[C].bind(xo, te.thread_axis("blockIdx.x"))
    s[C].bind(yo, te.thread_axis("blockIdx.y"))

    # bind the inner axes to threads
    s[C].bind(xi, te.thread_axis("threadIdx.x"))
    s[C].bind(yi, te.thread_axis("threadIdx.y"))

    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, B, C], simple_mode=True))
        print("=" * 100)

    return s, A, B, C

dev = tvm.cuda()
time, res, func, comp = benchmark_gemm_tvm(
    make_gemm_gpu_scheduler_v2, M, K, N, dev, a_np, w_np, num_runs=20, repeat=20
)
np.testing.assert_allclose(res, ref, rtol=1e-4)
print(f"[TVM v2] time: {time*1e3:.4f} ms")

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 32)
        threadIdx_x = T.launch_thread("threadIdx.x", 32)
        blockIdx_y = T.launch_thread("blockIdx.y", 16)
        threadIdx_y = T.launch_thread("threadIdx.y", 32)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y * 32 + threadIdx_y] = T.float32(0)
        for k in range(2048):
            A_1 = T.Buffer((2097152,), data=A.data)
            B_1 = T.Buffer((1048576,), data=B.data)
            C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y * 32 + threadIdx_y] = C_1[blockIdx_x * 16384 + threadIdx_x * 512 + blockIdx_y * 32 + threadIdx_y] 

In [17]:
# opt3: v2 + cache (with multi threads)
def make_gemm_gpu_scheduler_v3(M, K, N, verbose=True):
    k, s, A, B, C = base_declaration(M, K, N)
    block_x, block_y = 16, 16
    xo, xi = s[C].split(C.op.axis[0], factor=block_x)
    yo, yi = s[C].split(C.op.axis[1], factor=block_y)

    # split k
    tile_k = 8
    ko, ki = s[C].split(k, factor=tile_k)

    s[C].bind(xo, te.thread_axis("blockIdx.x"))
    s[C].bind(yo, te.thread_axis("blockIdx.y"))
    s[C].bind(xi, te.thread_axis("threadIdx.x"))
    s[C].bind(yi, te.thread_axis("threadIdx.y"))

    AA = s.cache_read(A, "shared", [C])
    BB = s.cache_read(B, "shared", [C])

    s[AA].compute_at(s[C], ko)
    s[BB].compute_at(s[C], ko)

    # multi threads for loading data
    # this increases performance a lot!
    AAxi, AAyi = s[AA].split(s[AA].op.axis[0], nparts=block_x)
    AAxx, AAxy = s[AA].split(s[AA].op.axis[1], nparts=block_y)
    s[AA].bind(AAxi, te.thread_axis("threadIdx.x"))
    s[AA].bind(AAxx, te.thread_axis("threadIdx.y"))

    BBxi, BByi = s[BB].split(s[BB].op.axis[0], nparts=block_x)
    BBxx, BBxy = s[BB].split(s[BB].op.axis[1], nparts=block_y)
    s[BB].bind(BBxi, te.thread_axis("threadIdx.x"))
    s[BB].bind(BBxx, te.thread_axis("threadIdx.y"))

    if verbose:
        print("=" * 100)
        print(tvm.lower(s, [A, B, C], simple_mode=True))
        print("=" * 100)

    return s, A, B, C


dev = tvm.cuda()
time, res, func, comp = benchmark_gemm_tvm(
    make_gemm_gpu_scheduler_v3, M, K, N, dev, a_np, w_np, num_runs=20, repeat=20
)
np.testing.assert_allclose(res, ref, rtol=1e-4)
print(f"[TVM v3] time: {time*1e3:.4f} ms")

# from tvm.script import ir as I
# from tvm.script import tir as T

@I.ir_module
class Module:
    @T.prim_func
    def main(A: T.Buffer((1024, 2048), "float32"), B: T.Buffer((2048, 512), "float32"), C: T.Buffer((1024, 512), "float32")):
        T.func_attr({"from_legacy_te_schedule": T.bool(True), "tir.noalias": T.bool(True)})
        blockIdx_x = T.launch_thread("blockIdx.x", 64)
        A_shared = T.allocate([128], "float32", "shared")
        B_shared = T.allocate([128], "float32", "shared")
        threadIdx_x = T.launch_thread("threadIdx.x", 16)
        blockIdx_y = T.launch_thread("blockIdx.y", 32)
        threadIdx_y = T.launch_thread("threadIdx.y", 16)
        C_1 = T.Buffer((524288,), data=C.data)
        C_1[blockIdx_x * 8192 + threadIdx_x * 512 + blockIdx_y * 16 + threadIdx_y] = T.float32(0)
        for k_outer in range(256):
            A_shared_1 = T.Buffer((128,), data=A_shared, scope="shared")
            with T.launch_thread("threadIdx.x", 16) as threadIdx_x_1:
       

In [13]:
import tvm
from tvm import te, autotvm
import numpy as np
import timeit
import os

@autotvm.template("gemm_gpu_autotvm")
def gemm_gpu(M, K, N, dtype="float32"):
    k, s, A, B, C = base_declaration(M, K, N)

    # Important: cache_write must be placed before any other transformations
    local_C = s.cache_write(C, "local")

    x, y = s[C].op.axis
    cfg = autotvm.get_config()

    cfg.define_knob("tile_x", [8, 16, 32])
    cfg.define_knob("tile_y", [8, 16, 32])
    cfg.define_knob("tile_k", [8, 16])

    xo, xi = s[C].split(x, factor=cfg["tile_x"].val)
    yo, yi = s[C].split(y, factor=cfg["tile_y"].val)


    s[C].bind(xo, te.thread_axis("blockIdx.x"))
    s[C].bind(yo, te.thread_axis("blockIdx.y"))
    s[C].bind(xi, te.thread_axis("threadIdx.x"))
    s[C].bind(yi, te.thread_axis("threadIdx.y"))

    s[local_C].compute_at(s[C], yi)
    k = s[local_C].op.reduce_axis[0]
    ko, ki = s[local_C].split(k, factor=cfg["tile_k"].val)

    AA = s.cache_read(A, "shared", [local_C])
    BB = s.cache_read(B, "shared", [local_C])

    s[AA].compute_at(s[local_C], ko)
    s[BB].compute_at(s[local_C], ko)

    cfg.define_knob("vectorize", [0, 1])
    if cfg["vectorize"].val:
        s[AA].vectorize(s[AA].op.axis[-1])
        s[BB].vectorize(s[BB].op.axis[-1])

    return s, [A, B, C]



task = autotvm.task.create("gemm_gpu_autotvm", args=(M, K, N, dtype), target="cuda")
print("Search space size:", len(task.config_space))

log_file = "gemm_gpu_autotvm.log"
tuner = autotvm.tuner.RandomTuner(task)
measure_option = autotvm.measure_option(
    builder=autotvm.LocalBuilder(),
    runner=autotvm.LocalRunner(repeat=3, min_repeat_ms=50, timeout=4)
)
num_trials = 30
tuner.tune(
    n_trial=num_trials,
    measure_option=measure_option,
    callbacks=[autotvm.callback.log_to_file(log_file)],
)

# apply the best configuration
with autotvm.apply_history_best(log_file):
    with tvm.target.Target("cuda"):
        s, args = gemm_gpu(M, K, N, dtype)
        func = tvm.build(s, args)

# verify and benchmark
a_np = np.random.rand(M, K).astype(dtype)
b_np = np.random.rand(K, N).astype(dtype)
c_np = np.zeros((M, N), dtype=dtype)

dev = tvm.cuda(0)
a_tvm = tvm.nd.array(a_np, dev)
b_tvm = tvm.nd.array(b_np, dev)
c_tvm = tvm.nd.array(c_np, dev)

func(a_tvm, b_tvm, c_tvm)
evaluator = func.time_evaluator(func.entry_name, dev, number=10, repeat=10)
time_cost = evaluator(a_tvm, b_tvm, c_tvm).mean * 1000  # convert to ms
print(f"AutoTVM tuned GEMM execution time: {time_cost:.4f} ms")
c_ref = np.matmul(a_np, b_np)
np.testing.assert_allclose(c_tvm.asnumpy(), c_ref, rtol=1e-4)


Search space size: 36
AutoTVM tuned GEMM execution time: 42.5615 ms


In [ ]:
# benchmark for numpy
def benchmark_gemm_numpy(a_np, w_np, num_runs=10):
    t0 = time.time()
    out = None
    for _ in range(num_runs):
        out = np.matmul(a_np, w_np)
    t1 = time.time()
    return (t1 - t0) / num_runs, out

# numPy baseline
numpy_time, numpy_out = benchmark_gemm_numpy(a_np, w_np, num_runs=10)
print(f"[NumPy]   time: {numpy_time*1e3:.4f} ms")
np.testing.assert_allclose(numpy_out, ref, rtol=1e-4)


[NumPy]   time: 74.9496 ms


In [ ]:
import torch
import time
import numpy as np


def benchmark_gemm_torch(M, K, N, a_np, w_np, num_runs=30, repeat=20, device="cuda"):
    a_torch = torch.tensor(a_np, dtype=torch.float32).to(device)
    w_torch = torch.tensor(w_np, dtype=torch.float32).to(device)

    for _ in range(10):
        _ = torch.matmul(a_torch, w_torch)

    torch.cuda.synchronize()

    times = []
    for r in range(repeat):
        start_time = time.time()
        for _ in range(num_runs):
            out = torch.matmul(a_torch, w_torch)
            torch.cuda.synchronize()
        end_time = time.time()
        times.append((end_time - start_time) / num_runs)

    avg_time = sum(times) / len(times)

    out_np = out.cpu().numpy()
    return avg_time, out_np

# torch cpu
cpu_time, cpu_res = benchmark_gemm_torch(M, K, N, a_np, w_np,
                                         num_runs=5, repeat=5, device="cpu")
np.testing.assert_allclose(cpu_res, ref, rtol=1e-4)
print(f"[PyTorch CPU] time: {cpu_time*1e3:.4f} ms")

# torch gpu
gpu_time, gpu_res = benchmark_gemm_torch(M, K, N, a_np, w_np,
                                        num_runs=20, repeat=20, device="cuda")
np.testing.assert_allclose(gpu_res, ref, rtol=1e-4)
print(f"[PyTorch CUDA] time: {gpu_time*1e3:.4f} ms")

[PyTorch CPU] time: 18.7425 ms
[PyTorch CUDA] time: 0.7027 ms


In [ ]:
# pytest
%pip install ipytest
import ipytest
ipytest.autoconfig()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 39.1 MB/s eta 0:00:00


In [ ]:
%%ipytest
import tvm
import torch
import pytest
import timeit
import numpy as np

make_gemm_gpu_scheduler = make_gemm_gpu_scheduler_v3

dev = tvm.cuda(0)

def ans_np(a, b):
    return np.matmul(a, b)


def make_func(M, K, N):
    s, A, B, O = make_gemm_gpu_scheduler(M, K, N)
    func = tvm.build(s, [A, B, O], "cuda")
    return func


def ans_torch(a_torch, b_torch):
    torch.cuda.synchronize()
    out = torch.mm(a_torch, b_torch)
    torch.cuda.synchronize()
    return out


@pytest.mark.parametrize('execution_number', range(5))
def test1_M1_N1_K2(execution_number):
    # Define dimension
    M = 1
    N = 1
    K = 2
    func = make_func(M, K, N)

    # Create random test data
    np.random.seed(seed=execution_number)
    a_np = np.random.rand(M, K).astype(np.float32)
    w_np = np.random.rand(K, N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M, N), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)


@pytest.mark.parametrize('execution_number', [1, 10, 100, 1000, 10000])
def test1_Mvar_N1024_K1024(execution_number):
    # Define dimension
    M = execution_number
    N = 1024
    K = 1024
    func = make_func(M, K, N)

    # Create random test data
    np.random.seed(seed=1024)
    a_np = np.random.rand(M, K).astype(np.float32)
    w_np = np.random.rand(K, N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M, N), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)


@pytest.mark.parametrize('execution_number', [1, 10, 100, 1000, 10000])
def test1_M1024_Nvar_K1024(execution_number):
    # Define dimension
    M = 1024
    N = execution_number
    K = 1024
    func = make_func(M, K, N)

    # Create random test data
    np.random.seed(seed=1024)
    a_np = np.random.rand(M, K).astype(np.float32)
    w_np = np.random.rand(K, N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M, N), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)


@pytest.mark.parametrize('execution_number', [1, 10, 100, 1000, 10000])
def test1_M1024_N1024_Kvar(execution_number):
    # Define dimension
    M = 1024
    N = 1024
    K = execution_number
    func = make_func(M, K, N)

    # Create random test data
    np.random.seed(seed=1024)
    a_np = np.random.rand(M, K).astype(np.float32)
    w_np = np.random.rand(K, N).astype(np.float32)
    b_np = ans_np(a_np, w_np)

    a = tvm.nd.array(a_np, dev)
    w = tvm.nd.array(w_np, dev)
    b = tvm.nd.array(np.zeros((M, N), dtype='float32'), dev)
    func(a, w, b)
    b_out = b.numpy()

    assert b_np.shape == b_out.shape, \
        "Shape mismatch: " + str(b_np.shape) + "\t" + str(b_out.shape)
    assert np.allclose(b_np, b_out), "Value mismatch: %s %s" % (b_np, b_out)



....................                                                                         [100%]
20 passed in 19.22s
